# Use case 4: corner reflector

This section explains how to identify bursts in which the corner reflector (CR), having known geodetic coordinates, is visible, and how to get corrections at CR location.

## Notebook setup

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../..')

In [4]:
import numpy as np
from scipy.interpolate import interp2d
from matplotlib import pyplot as plt

In [5]:
import s1etad
from s1etad import Sentinel1Etad, ECorrectionType

## Searching bursts in which the CR is present

Load the S1-ETAD product.

In [6]:
filename = '../../sample-products/S1B_IW_ETA__AXDH_20200127T113414_20200127T113858_020002_025D72_0096.SAFE'
eta = Sentinel1Etad(filename)

The CR position has been chosed to be in the overlab region of different bursts and swaths.

In [7]:
from shapely.geometry import Point

lat0 = +70.13  # 70° 7'48.65"N
lon0 = -76.19  # 76°11'41.55"W
h0 = 366

cr = Point(lon0, lat0)

Query for burst cobering the CR.

In [8]:
selection = eta.query_burst(geometry=cr)
selection

,bIndex,pIndex,sIndex,productID,swathID,azimuthTimeMin,azimuthTimeMax
274,273,10,3,S1B_IW_SLC__1ADH_20200127T113802_20200127T1138...,IW3,2020-01-27 11:38:24.612132,2020-01-27 11:38:27.806467
265,275,10,2,S1B_IW_SLC__1ADH_20200127T113802_20200127T1138...,IW2,2020-01-27 11:38:26.396355,2020-01-27 11:38:29.619467
295,276,11,3,S1B_IW_SLC__1ADH_20200127T113827_20200127T1138...,IW3,2020-01-27 11:38:27.374800,2020-01-27 11:38:30.569134


## Get corrections at CR location

Reatrieve the first burst.

In [9]:
burst = next(eta.iter_bursts(selection))

Get RADAR coordinates (range and azimuth time) of the CR.

In [10]:
tau0, t0 = burst.geodetic_to_radar(lat0, lon0, h0)
tau0, t0

ValueError: A value in x_new is below the interpolation range.

Get image coordinates (line and sample) of the CR.

**NOTE**: image coordinates area flaoting point numbers including pixel fractions.

In [ ]:
line, sample = burst.radar_to_image(t0, tau0)
print(f'Line {line} of {burst.lines}, sample {sample} of {burst.samples}.')

Get the correction.

In [ ]:
correction = burst.get_correction(s1etad.ECorrectionType.SUM, meter='True')
data = correction['x']  # summ of corrections in the range direction

Interpolate the correction at the specified RADAR coordinates (tau0, t0).

In [ ]:
azimuth_time, range_time = burst.get_burst_grid()

interpolator = interp2d(range_time, azimuth_time, data)

value_at_radar_coordinates = interpolator(tau0, t0)

print(f'The correction value at RADAR coordinates (tau0, t0) = ({tau0}, {t0}) is {value_at_radar_coordinates}')    

Interpolate the correction at the specified image coordinates (line, sample).

In [ ]:
xaxis = np.arange(burst.samples)
yaxis = np.arange(burst.lines)

interpolator = interp2d(xaxis, yaxis, data)

value_at_image_coordinates = interpolator(sample, line)

print(f'The correction value at image coordinates (sample, line) = ({sample}, {line}) is {value_at_image_coordinates}') 

## Putting all together

In [ ]:
from matplotlib.patches import Rectangle

fig, ax = plt.subplots(nrows=len(selection), ncols=1, figsize=[13, 8])

for loop, burst in enumerate(eta.iter_bursts(selection)):
    # get RADAR and image coordinates
    tau0, t0 = burst.geodetic_to_radar(lat0, lon0)
    line0, sample0 = burst.radar_to_image(t0, tau0)    
    print("xy  ", burst.swath_id, burst.burst_index, line0, sample0, burst.radar_to_geodetic(tau0, t0))
    print("time", burst.swath_id, burst.burst_index, tau0, t0, burst.radar_to_geodetic(tau0, t0))
    
    # correction
    cor = burst.get_correction(s1etad.ECorrectionType.SUM, meter='True')

    ax[loop].imshow(cor['x'], aspect='auto')
    rec_half_size = 1
    p = Rectangle((sample0 - rec_half_size, line0 - rec_half_size),
                  width=rec_half_size*2+1, height=rec_half_size*2+1,
                  color='red', fill=True)
    ax[loop].add_patch(p)

    # get the range and azimuth time axes
    azimuth_time, range_time = burst.get_burst_grid()

    # interpolate at the desired working (RADAR) coordinates
    f_t  = interp2d(range_time, azimuth_time, cor['x'])

    # get the image (lines and samples) axes
    yaxis = np.arange(azimuth_time.size)
    xaxis = np.arange(range_time.size)
    
    # interpolate at the desired working (image) coordinates
    f_ij = interp2d(xaxis, yaxis, cor['x'])
    
    print(f"Interpoaltion by array coordinate {f_ij(sample0, line0)} or time {f_t(tau0,t0)} should be the same")
    print(f"The total correction at lat/lon {lat0, lon0} is  {f_ij(sample0, line0)} m in range")
    print()

In [ ]:
burst.radar_to_geodetic(tau0, t0) 

In [ ]:
cor